In [14]:
import pandas as pd
from mlforecast import MLForecast
from xgboost import XGBRegressor
from utilsforecast.plotting import plot_series
from matplotlib import pyplot as plt
from window_ops.rolling import rolling_mean, rolling_std, rolling_min, rolling_max
from mlforecast.target_transforms import Differences
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np
from scipy import signal
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from prophet import Prophet
from sklearn.base import BaseEstimator, RegressorMixin

In [15]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')


In [28]:
df=pd.read_csv('MLTempDataset.csv')
df.drop(columns=['Unnamed: 0',],inplace=True)#'Datetime1'
df1=df.copy()
df1['unique_id']='h1'
# df1['unique_id']=
df1['ds']=pd.to_datetime(df['Datetime']).dt.date
df1['y']=df['DAYTON_MW']
df1.drop(columns=['Datetime','DAYTON_MW'],inplace=True)
df1.head()

,Datetime1,unique_id,ds,y
0,0,h1,2022-01-04,20.867
1,1,h1,2022-01-04,21.000
2,2,h1,2022-01-04,20.867
3,3,h1,2022-01-04,20.650
4,4,h1,2022-01-04,20.400


In [34]:
df1['ds']=pd.to_datetime(df1['ds'])

In [36]:
df1=df1.groupby(df1['ds'].dt.date)['y'].sum().reset_index()
df1.head()

,ds,y
0,2022-01-04,505.550
1,2022-01-05,369.015
2,2022-01-06,470.184
3,2022-01-07,373.068
4,2022-01-08,378.718


In [38]:
df2=df1.copy()
df2['DS']=df2['ds']
df2['Y']=df2['y']
df2=df2.drop(columns=['ds','y'])
df2.head()

,DS,Y
0,2022-01-04,505.550
1,2022-01-05,369.015
2,2022-01-06,470.184
3,2022-01-07,373.068
4,2022-01-08,378.718


In [39]:
train=df1[:5340]
test=df1[5340:]
train.shape,test.shape

((279, 2), (0, 2))

In [40]:
class ProphetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **prophet_params):
        self.prophet_params = prophet_params
        self.model = None

    def fit(self, X, y):
        df = X.copy()
        df['y'] = y
        df['ds'] = df.index
        self.model = Prophet(**self.prophet_params)
        self.model.fit(df)
        return self

    def predict(self, X):
        df = X.copy()
        df['ds'] = df.index
        return self.model.predict(df)['yhat'].values

In [41]:
m=Prophet()
m.fit(train)

08:46:49 - cmdstanpy - INFO - Chain [1] start processing
08:46:49 - cmdstanpy - INFO - Chain [1] done processing


In [42]:
future = m.make_future_dataframe(periods=1440)
future.tail()

,ds
1714,2026-09-14
1715,2026-09-15
1716,2026-09-16
1717,2026-09-17
1718,2026-09-18


In [11]:
model=XGBRegressor(random_state=990,learning_rate=0.01,n_estimators=1000,max_depth=20,reg_lambda=0.2)
ftsc=MLForecast(
    models={
        # 'xgb':model,
        'prophet':ProphetWrapper(),
    },
    freq='H',
    # lags=[24,168 ,350,720],
    # lag_transforms={
    #     168:[(rolling_mean,168),(rolling_std,168),(rolling_min,168),(rolling_max,168)],
    #     350:[(rolling_mean,350),(rolling_std,350),(rolling_min,350),(rolling_max,350)],
    #     720:[(rolling_mean,720),(rolling_std,720),(rolling_min,720),(rolling_max,720)]
    # },
    # target_transforms=[Differences([168,350,720])],
    # date_features=["day",'hour','weekday','month','year','dayofyear','quarter'],
)
ftsc.fit(train,id_col='unique_id',time_col='ds',target_col='y',static_features=['Datetime1'])
# min_child_weight=
# ,gamma=
# ,subsample=
# objective="reg:squarederror",

In [9]:
preds=ftsc.predict(1336)
preds.head()

In [ ]:
mse_test = mean_squared_error(test['y'], preds['xgb'])
mse_test